# Assistants API를 사용하여 챗봇 구축

OpenAI의 Assistants API를 사용하여 맞춤형 챗봇을 만드는 방법에 대해 알아볼 것입니다. 이 프로젝트는 대화 방식으로 사용자 입력을 이해하고 응답하는 OpenAI 언어 모델의 기능을 선보일 것입니다. 다양한 주제에 대해 사용자와 대화하고 관련성이 있고 상황에 맞는 응답을 제공할 수 있는 챗봇을 만드는 방법을 배우게 됩니다.

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

## request 보내기

In [4]:
response = client.chat.completions.create(
    model = Model,
    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "세계에서 가장 높은 빌딩은 어느 것인가요?"}
    ],
    temperature=0.9,
    max_tokens=300
)

response.choices[0].message.content

'현재 세계에서 가장 높은 빌딩은 두바이에 위치한 부르즈 할리파(Burj Khalifa)입니다. 이 건물은 828미터(2,717피트)로, 163층으로 이루어져 있습니다. 2010년에 완공된 부르즈 할리파는 현대 건축의 상징적인 예로 꼽히며, 많은 관광객들이 방문하는 명소입니다.'

# OpenAI API를 사용하여 맞춤형 챗봇 구축

API는 매번 새로운 session 이 시작되므로 중간의 대화 내용을 기억하지 못합니다. 해결 방법은,  
1. 주고 받은 대화 내용을 메모리에 저장 시키면 이전의 대화 내용을 기억할 수 있습니다.
2. Assistant API의 Thread 기능을 이용하여 중간 대화 history를 저장할 수 있습니다.

```
(withoug Assistant) - 주고 받은 내용을 메모리에 저장 
사용자 입력: My name is Gildong. What is my name?
Assistant:  Your name is Gildong.
사용자 입력: What is my name?
Assistant:  I apologize, but I'm not able to determine your name as I don't have access to your personal information.
사용자 입력: quit
```

In [5]:
# 대화 기록을 저장할 메모리 버퍼 초기화 (시스템 메시지 포함)
memory_buffer = [{"role": "system", "content": "You are a helpful assistant."},]

# Assistant 없이 챗봇 구축 (사용자 입력을 직접 처리)
while True:
    user_input = input("\n사용자 입력: ")  # 사용자 입력 받기
    if user_input == "quit":  # "quit" 입력 시 종료
        break

    # 사용자 입력을 메모리 버퍼에 추가
    memory_buffer.append({"role": "user", "content": user_input})
    
    # penAI API를 사용하여 챗봇 응답 생성
    response = client.chat.completions.create(
        model = Model,  # 사용할 모델 (예: "gpt-4-turbo")
        messages = memory_buffer,  # 대화 기록 전달
        temperature=0.9,  # 창의적인 응답을 위해 온도값 설정 (높을수록 다양성 증가)
        max_tokens=300  # 최대 응답 토큰 수 제한
    )

    # Assistant의 응답 출력
    print("Assistant: ", response.choices[0].message.content)


사용자 입력:  안녕 내이름은 길동이야


Assistant:  안녕하세요, 길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?



사용자 입력:  내 이름이 뭐라고 했지?


Assistant:  당신의 이름은 길동이시라고 하셨습니다.



사용자 입력:  quit


### Assistants API 이용 챗봇 구축

In [9]:
# 1단계 - Assistant 생성
assistant = client.beta.assistants.create(
  name="Fruit Adviser",
  instructions="너는 과일 지식이 풍부한 과일 전문가야. 나의 질문에 친절히 답변을 해주는 Chatbot이야.",
  model=Model,
)

In [10]:
# 2단계 - Thread 생성
thread = client.beta.threads.create()

In [11]:
from typing_extensions import override
from openai import AssistantEventHandler

# 먼저, 이벤트 핸들러 클래스(EventHandler)를 생성하여
# 응답 스트림에서 이벤트를 어떻게 처리할지 정의합니다.

class EventHandler(AssistantEventHandler):
  # 텍스트 생성이 완료되었을 때 호출되는 메서드
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  # 텍스트 생성 중간에 호출되는 메서드
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

# Run을 생성하고 응답을 스트리밍합니다.
with client.beta.threads.runs.stream(
  thread_id=thread.id,          # 실행할 스레드의 ID
  assistant_id=assistant.id,    # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 사용자에게 프리미엄 계정이 있습니다",  # 어시스턴트에게 제공할 지시사항
  event_handler=EventHandler(),     # EventHandler 클래스 인스턴스를 이벤트 핸들러로 사용
) as stream:
  stream.until_done()           # 스트림이 완료될 때까지 대기


assistant > 안녕하세요, 고객님! 무엇을 도와드릴까요?

In [12]:
while True:
    user_input = input("사용자 입력: ")
    if user_input == "quit":
        break

    # 3단계 - Thread에 Message 추가
    message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=user_input
    )

    # 4단계 - Run 생성
    with client.beta.threads.runs.stream(
      thread_id=thread.id,
      assistant_id=assistant.id,
      instructions=user_input,
      event_handler=EventHandler(),
    ) as stream:
      stream.until_done()

사용자 입력:  안녕. 내이름은 길동이야



assistant > 안녕하세요, 길동님! 반갑습니다. 어떤 이야기를 나누고 싶으신가요?

사용자 입력:  내 이름이 뭐라고 했지?



assistant > 당신의 이름은 길동이시라고 말씀하셨습니다. 맞나요?

사용자 입력:  quit


### 문서/텍스트를 기반으로 챗봇이 답변하도록 하기

문서/텍스트 내용을 API 호출시 넘겨주면 우리가 원하는 답변을 유도할 수 있습니다.

In [13]:
dataset="""Document content:
질문: UNDP의 eRecruit 시스템은 무엇입니까? 답변: UNDP의 eRecruit 시스템은 지원자가 하나 이상의 광고된 UNDP 채용 공고에 지원서를 제출하기 위해 정기적으로 업데이트할 수 있는 개인 프로필을 만들 수 있는 온라인 시스템입니다.
질문: UNDP의 eRecruit 시스템에 어떻게 액세스합니까? 답변: UNDP의 eRecruit 시스템은 다음 링크를 통해 접속할 수 있습니다: https://undpcareers.partneragency.org/erecruit.html
질문: UNDP의 eRecruit 시스템을 사용하여 지원하는 절차는 무엇입니까? 답변: 지원 절차 지원 절차
질문: UNDP의 eRecruit 시스템을 사용하여 온라인으로 지원해야 합니까? 답변: 모든 지원서는 UNDP의 eRecruit 시스템을 사용하여 온라인으로 제출해야 합니다. 오프라인 서면 지원서나 이메일을 통한 지원서는 접수되지 않습니다.
질문: UNDP의 eRecruit 시스템과 호환되는 브라우저는 무엇입니까? 답변: UNDP의 eRecruit 시스템은 Google Chrome, Internet Explorer 6 이상에 최적화되어 있습니다. 호환성 보기 모드를 사용하려면 Internet Explorer 9을 사용해야 합니다. 신청서가 성공적으로 제출되었는지 확인하려면 다음 브라우저 중 하나를 사용하는 것이 좋습니다.
질문: 내 프로필에 로그인하는 데 문제가 있습니다. 브라우저가 응답하지 않습니다. 어떻게 해야 하나요? 답변: 구인 신청을 위해 UNDP eRecruit 프로필에 로그인하는 데 문제가 있는 경우 이는 여러 가지 이유에서 비롯될 수 있으며 그 중 일부는 인터넷 연결과 같은 UNDP의 통제 범위를 벗어납니다. 그러나 이러한 유형의 문제를 해결하려면 다음 지침을 따르는 것이 좋습니다. • 권장 브라우저(및 버전)를 사용하고 있는지 확인하십시오. • 캐시/브라우저 기록을 지웁니다. http://www.refreshyourcache.com/en/home에서 브라우저 기록을 지우는 방법에 대한 정보를 찾을 수 있습니다. 이 작업을 수행하기 전에 브라우저에서 캐시를 지울 때의 결과를 알고 있는지 확인하십시오.
질문: UNDP의 eRecruit 시스템을 사용하는 데 도움이 필요하면 어떻게 합니까? 답변: UNDP의 eRecruit 시스템 사용에 대한 일반적인 질문이나 지원이 필요한 경우 헬프데스크(https://info.undp.org/sas/erecruit/Assets/HelpDesk.aspx)에 문의하세요.
질문: 왜 등록해야 합니까? 답변: 모든 지원자는 먼저 UNDP의 eRecruit 시스템에 등록해야 합니다. 등록이 완료되면 개인 정보를 입력하고 광고된 채용 공고에 지원할 수 있도록 개인 계정이 생성됩니다.
질문: 등록할 때 사용자 이름으로 무엇을 사용해야 합니까? 답변: UNDP eRecruit 시스템에 등록할 때 유효한 이메일 주소를 사용자 이름으로 사용하는 것이 좋습니다.
질문: 어떤 비밀번호 형식이 허용되나요? 답변: UNDP는 강력한 비밀번호 사용을 권장합니다. 비밀번호는 최소 8자 이상이어야 하며 문자와 숫자를 조합해야 합니다.
질문: 비밀번호를 어떻게 변경할 수 있나요? 답변: 시스템에 로그인한 후 '개인 정보' 링크에서 '비밀번호 변경' 옵션을 선택하세요. '사용자 이름 또는 비밀번호를 잊으셨나요?' 링크를 클릭하고 지침에 따라 잊어버린 비밀번호를 검색하세요.
질문: 비밀번호를 잊어버렸습니다. 어떻게 해야 하나요? 답변: 비밀번호를 잊어버린 경우, 사용자 ID 또는 비밀번호 찾기 링크를 클릭하고 두 가지 옵션 중 하나를 완료하세요.
질문: 내 데이터는 안전합니까? 답변: UNDP의 eRecruit 데이터는 개인 보안 데이터베이스에 저장되며 UNDP는 데이터의 소유자입니다. 이 시스템은 UNDP의 엄격한 보안 요구 사항을 충족합니다.
질문: 각 섹션의 모든 정보를 작성해야 합니까? 답변: 별표(*)가 표시된 모든 항목은 필수 정보이며 각 섹션별로 작성해야 합니다.
질문: 한 세션에서 모든 정보를 완료해야 합니까? 답변: 지원 과정 중 언제든지 지원서를 저장하고 나중에 계속할 수 있습니다. 섹션을 완료할 때 정보가 손실되지 않도록 정기적으로 저장 버튼을 사용하는 것이 좋습니다.
질문: 달력 기능을 사용하여 날짜를 어떻게 선택합니까? 답변: 날짜는 두 가지 방법으로 선택할 수 있습니다. A) dd/mm/yyyy 형식을 사용하여 필드에 날짜를 직접 입력하거나 B) 달력 아이콘을 클릭하여 날짜를 선택하고 연도, 월 및 일을 선택할 수 있습니다.
질문: 관련 정보가 드롭다운 옵션에 포함되어 있지 않으면 어떻게 해야 합니까? 답변: 사용 가능한 드롭다운 옵션 중에서 선택해야 합니다. 귀하의 개인 정보, 기술 및 경험에 가장 가까운 옵션을 선택하십시오.
질문: 조회 기능을 어떻게 사용합니까? 답변: 조회 기능을 사용하려면 돋보기 아이콘을 클릭하세요. 그런 다음 해당 필드에 전체 또는 부분 값을 입력하고 조회 버튼을 클릭합니다. 마지막으로 검색 결과에서 적절한 값을 선택하세요.
질문: 맞춤법 검사 기능을 사용할 수 있나요? 답변: 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 클릭 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 사전을 클릭하세요.
"""

In [14]:
# 1단계 - Assistant 생성
assistant = client.beta.assistants.create(
  name="콜센터 도우미",
  instructions="""
        콜센터 응답자 역할을 해주시기 바랍니다. 당신의 이름은 "콜센터 도우미"입니다.
        당신은 주어진 정보에 대한 답변을 나에게 제공할 것입니다. 답변이 포함되지 않은 경우 "죄송합니다. 정보가 없습니다."라고 말하세요. 그 후에는 중지하세요.
        정보에 관한 질문이 아니면 답변을 거부하세요.
        """,
  model=Model,
)

In [15]:
# 2단계 - Thread 생성
thread = client.beta.threads.create()

In [16]:
# 3단계 - Thread에 Message 추가
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=dataset
)

In [18]:
while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() == "quit":
        break

    # 3단계 - Thread에 Message 추가
    message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=user_input
    )

    # 4단계 - Run 생성
    with client.beta.threads.runs.stream(
      thread_id=thread.id,
      assistant_id=assistant.id,
      instructions="사용자를 고객님이라고 부르세요.",  # 어시스턴트에게 제공할 지시사항
      event_handler=EventHandler(),
    ) as stream:
      stream.until_done()

사용자 입력:  UNDP는 어떤 비밀번호 형식을 사용하나요



assistant > 고객님, UNDP는 비밀번호에 대해 다음과 같은 형식을 요구합니다:

- 최소 8자 이상이어야 합니다.
- 문자와 숫자를 조합해야 합니다.

비밀번호를 설정할 때 이러한 조건을 충족해야 하므로 참고하시기 바랍니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!

사용자 입력:  QUIT


### <<대화 예시 >>
사용자 입력: 맞춤법 검사 기능을 사용할 수 있나요  
Assistant:  네, 맞춤법 검사 기능을 사용할 수 있습니다. 어떤 문장을 맞춤법 검사해드릴까요?  

사용자 입력: UNDP는 어떤 비밀번호 형식을 요구하나요  
Assistant:  UNDP는 다음과 같은 비밀번호 형식을 요구합니다:
- 최소 8자 이상의 길이
- 대문자와 소문자를 혼합하여 사용
- 숫자와 특수문자를 혼합하여 사용  

사용자 입력: 각 섹션의 모든 정보를 제공해야하나요  
Assistant:  UNDP에 따르면, 각 섹션에서 요구하는 정보는 다를 수 있습니다. 일반적으로, UNDP 신청서나 등록 절차에서는 다음과 같은 정보를 제공해야 할 수 있습니다:

1. 개인 정보: 이름, 생년월일, 성별, 국적 등 개인 신상정보
2. 연락처 정보: 전화번호, 이메일 주소, 주소 등 연락처 정보
3. 학력 및 경력: 학력 수준, 전공, 졸업 일자, 이전 직장 경력 등
4. 기술 및 자격증: 특정 기술 또는 자격증 보유 여  

사용자 입력: 강남에서 남산에 가는 교통편을 알려주세요  
Assistant:  강남에서 남산으로 가는 교통편은 다양합니다. 여러 가지 옵션 중에서 몇 가지를 알려드리겠습니다:

1. 지하철:
   - 강남역에서 3호선을 타고 충무로역으로 이동한 후, 3호선에서 4호선으로 환승하여 회현역에서 1호선으로 환승합니다. 그리고 남산역에서 하차하면 됩니다.

2. 버스:
   - 강남역 혹은 강남역 근처에서 남산 방향으로 가는 버스를 탈 수 있습니다. 예를 들어, 402, 405, 406  

사용자 입력: stop
Assistant:  알겠습니다. 추가로 도움이 필요하시면   
언제든지 말씀해주세요. 좋은 하루 되세요!  

사용자 입력: quit